<a href="https://colab.research.google.com/github/devmams/american-vote/blob/master/american_vote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Récupération des données

In [0]:
!wget https://raw.githubusercontent.com/devmams/machinelearning/master/americanvote/data.txt

--2020-05-22 00:18:15--  https://raw.githubusercontent.com/devmams/machinelearning/master/americanvote/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [text/plain]
Saving to: ‘data.txt.1’

data.txt.1          100%[===================>]  17.75K  --.-KB/s    in 0.004s  

2020-05-22 00:18:15 (4.85 MB/s) - ‘data.txt.1’ saved [18171/18171]



# Pré-traitement



In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
fichier = open("data.txt", "r")
lignes = fichier.readlines()

In [0]:
# republican = 1 ,democrat = 0 ,y = 1, n = -1 et ? = 0
def choixVote(ch):
    if ch == 'republican':
        return 1
    elif ch == 'democrat':
        return 0

    elif ch == 'y':
        return 1
    elif ch == 'n':
        return -1
    else:
        return 0

In [0]:
Vote = []
i = 0
for ligne in lignes:
    ligne = ligne.replace('\n','')
    t = ligne.split(',')
    for i in range(0,len(t)):
        Vote.append(choixVote(t[i]))

Vote = np.reshape(Vote,(-1,17))
np.random.seed(100)
np.random.shuffle(Vote)

In [0]:
Target = Vote[:,0]
Data = Vote[:,1:]

In [0]:
nbExempleTrain = 335
TrainData = Data[0:nbExempleTrain,:]
TrainTarget = Target[0:nbExempleTrain]
TestData = Data[nbExempleTrain:,:]
TestTarget = Target[nbExempleTrain:]

# Apprentissage

In [0]:
import torch
import torch.utils.data as utils
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import random
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [0]:
torch.manual_seed(2)

NameError: ignored

In [0]:
torch.set_default_tensor_type(torch.FloatTensor)

train_data_tensor = torch.tensor(TrainData)
train_target_tensor = torch.tensor(TrainTarget) # transform to torch tensors

test_data_tensor = torch.tensor(TestData) # transform to torch tensors
test_target_tensor = torch.tensor(TestTarget) # transform to torch tensors

train_dataset = utils.TensorDataset(train_data_tensor,train_target_tensor) # create your datset
test_dataset = utils.TensorDataset(test_data_tensor,test_target_tensor) # create your datset

batch_size = 1
# Set the training loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
# Set the testing loader
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Paramètres du modèle

In [0]:
weights = torch.randn(16, 1, requires_grad=True)
b1 = torch.randn(1, requires_grad=True)

In [0]:
it = 0
loss_mse = nn.MSELoss()
for batch_idx, (inputs, targets) in enumerate(train_loader):
    # Be sure to start the loop with zeros grad
    if weights.grad is not None:
        weights.grad.zero_()

    if b1.grad is not None:
      b1.grad.zero_()

    inputs = inputs.float()
    targets = targets.float()
    outputs = torch.matmul(inputs, weights) + b1
    outputs = torch.sigmoid(outputs)
    targets = targets.view_as(outputs)
    
    loss = loss_mse(outputs, targets)
    loss.backward()
    
    with torch.no_grad():
      weights -= 0.1*weights.grad
      b1 -= 0.1*b1.grad

    it += 1
    if it % 10 == 0:
      print("\rLoss : {}".format(loss), end="")
      predictions, targets = predict(weights,b1,test_loader)
      print(" | Accuracy : ", accuracy_score(targets, predictions)) 


Loss : 0.7204058766365051 | Accuracy :  0.74
Loss : 0.35589414834976196 | Accuracy :  0.76
Loss : 0.02778741717338562 | Accuracy :  0.78
Loss : 0.9252195954322815 | Accuracy :  0.78
Loss : 1.0038837672254886e-06 | Accuracy :  0.78
Loss : 0.18206742405891418 | Accuracy :  0.78
Loss : 1.0503016483198735e-06 | Accuracy :  0.78
Loss : 0.8814020156860352 | Accuracy :  0.79
Loss : 0.010065912269055843 | Accuracy :  0.8
Loss : 2.901505649788305e-05 | Accuracy :  0.8
Loss : 1.858751056715846e-05 | Accuracy :  0.82
Loss : 0.7760655879974365 | Accuracy :  0.83
Loss : 0.18622919917106628 | Accuracy :  0.83
Loss : 0.3798208236694336 | Accuracy :  0.84
Loss : 7.658655931663816e-08 | Accuracy :  0.84
Loss : 6.066415608074749e-06 | Accuracy :  0.85
Loss : 0.005354143213480711 | Accuracy :  0.86
Loss : 1.0849063301066053e-06 | Accuracy :  0.87
Loss : 0.004046598449349403 | Accuracy :  0.87
Loss : 0.49046939611434937 | Accuracy :  0.87
Loss : 0.005225390195846558 | Accuracy :  0.87
Loss : 0.00033433866

In [0]:
def predict(weights, b1, test_loader):
    predictions = []
    targets = []
    for batch_idx, (inputs, t) in enumerate(test_loader):
        inputs = inputs.float()
        t = t.float()
        outputs = torch.matmul(inputs, weights) + b1
        outputs = torch.sigmoid(outputs)
        pred = outputs.round()
        predictions.append(pred.item())
        targets.append(t.item())
       
    return np.fromiter(predictions, dtype=int), np.fromiter(targets, dtype=int)


In [0]:
predictions, targets = predict(weights,b1,test_loader)
print(predictions[0:10])
print(targets[0:10])
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(targets, predictions))
print(classification_report(targets, predictions))
print(accuracy_score(targets, predictions))

[1 1 0 0 0 1 0 0 0 1]
[0 1 0 0 0 1 0 0 0 1]
[[50  7]
 [ 4 39]]
              precision    recall  f1-score   support

           0       0.93      0.88      0.90        57
           1       0.85      0.91      0.88        43

    accuracy                           0.89       100
   macro avg       0.89      0.89      0.89       100
weighted avg       0.89      0.89      0.89       100

0.89


In [0]:
def test(weights, b1, test_loader):
    test_size = len(test_loader.dataset)
    correct = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs = inputs.float()
        targets = targets.float()
        outputs = torch.matmul(inputs, weights) + b1
        outputs = torch.sigmoid(outputs)
        pred = outputs.round()
        n_correct = pred.eq(targets.view_as(pred)).sum().item()
        correct += n_correct

    acc = correct / test_size
    print(" | Accuracy : ", acc, "| Bad predictions : ", test_size-correct, " / ", test_size)

test(weights, b1, test_loader)

 | Accuracy :  0.89 | Bad predictions :  11  /  100


In [0]:
# from sklearn.svm import LinearSVC

# clf_SVC = LinearSVC()

# clf_SVC.fit(X_train, y_train)

# y_pred_2 = clf_SVC.predict(X_test)
# clf_SVC.score(X_test, y_test)
# print("---- %d", (X_test.shape[0],(y_test != y_pred_2).sum()))
# print(confusion_matrix(y_test,y_pred_2))
# print(classification_report(y_test,y_pred_2))
# print(accuracy_score(y_test, y_pred_2))